# 01. Basic Example
A simple example on how to use the FDTD Library

## Imports

In [1]:
import matplotlib.pyplot as plt
import sys
sys.path.append('/home/bij/Projects/fdtd/')
import fdtd
import fdtd.backend as bd

## Set Backend

In [2]:
fdtd.set_backend("torch")

## Constants

In [3]:
WAVELENGTH = 1550e-9
SPEED_LIGHT: float = 299_792_458.0  # [m/s] speed of light

## Simulation

create FDTD Grid

In [4]:
grid = fdtd.Grid(
    #(8.5e-6, 8.5e-6, 8.5e-6), 
    #(1.5e-5, 1.5e-5, 1.5e-5),
    #(3.5e-6, 3.5e-6, 3.5e-6),
    #(1.5e-6, 1.5e-6, 1.5e-6),
    #(3.5e-6, 1.0e-6, 1.0e-6),
    #(3.0e-6, 0.75e-6, 0.75e-6),
    #(3.0e-5, 0.5e-6, 0.5e-6),
    (3.0e-5, 0.75e-6, 0.75e-6),
    grid_spacing=0.01 * WAVELENGTH,
    permittivity=1.0,
    permeability=1.0,
)
grid.shape

(1935, 48, 48)

boundaries

In [5]:
# grid[0, :, :] = fdtd.PeriodicBoundary(name="xbounds")
grid[0:10, :, :] = fdtd.PML(name="pml_xlow")
grid[-10:, :, :] = fdtd.PML(name="pml_xhigh")

# grid[:, 0, :] = fdtd.PeriodicBoundary(name="ybounds")
grid[:, 0:10, :] = fdtd.PML(name="pml_ylow")
grid[:, -10:, :] = fdtd.PML(name="pml_yhigh")

#grid[:, :, 0] = fdtd.PeriodicBoundary(name="zbounds")
grid[:, :, 0:10] = fdtd.PML(name="pml_zlow")
grid[:, :, -10:] = fdtd.PML(name="pml_zhigh")

sources

In [6]:
grid[11, :, :] = fdtd.PlaneSource(
    period=WAVELENGTH / SPEED_LIGHT, name="planesource_0", polarization='y'
)
grid[15, :, :] = fdtd.PlaneSource(
    period=WAVELENGTH / SPEED_LIGHT, name="planesource_1", polarization='y'
)


detectors

In [7]:
grid[:, grid.shape[1]//2, grid.shape[2]//2] = fdtd.LineDetector(name="detector")

## Run simulation

In [ ]:
grid.run(3000, progress_bar=True)

  7%|███████▌                                                                                                          | 200/3000 [01:50<26:19,  1.77it/s]

## Visualization

In [ ]:
fig, axes = plt.subplots(2, 3, squeeze=False)
titles = ["Ex: xy", "Ey: xy", "Ez: xy", "Hx: xy", "Hy: xy", "Hz: xy"]

fields = bd.stack(
    [
        grid.E[:, :, 0, 0],
        grid.E[:, :, 0, 1],
        grid.E[:, :, 0, 2],
        grid.H[:, :, 0, 0],
        grid.H[:, :, 0, 1],
        grid.H[:, :, 0, 2],
    ]
)

m = max(abs(fields.min().item()), abs(fields.max().item()))

for ax, field, title in zip(axes.ravel(), fields, titles):
    ax.set_axis_off()
    ax.set_title(title)
    ax.imshow(bd.numpy(field), vmin=-m, vmax=m, cmap="RdBu")

plt.show()

In [ ]:
plt.figure()
grid.visualize(z=0)

In [ ]:
Hs = bd.stack(grid.detectors[0].H, axis=0)
Es = bd.stack(grid.detectors[0].E, axis=0)
print(Hs.shape, Es.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_EH_3d_vectors(E_vectors, H_vectors, scale=1.0):
    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(111, projection='3d')
    
    def normalize_and_plot(vectors, color):
        # Normalize vectors
        normalized_vectors = vectors / np.max([np.linalg.norm(v) for v in vectors])

        # Extract x, y, z components of each normalized vector
        x = [v[0] for v in normalized_vectors]
        y = [v[1] for v in normalized_vectors]
        z = [v[2] for v in normalized_vectors]

        # Plot the vectors
        ax.quiver(np.arange(len(x))/len(x), 0, 0, x, y, z, length=scale, arrow_length_ratio=0.05, colors=color)

    normalize_and_plot(E_vectors, color='red')
    normalize_and_plot(H_vectors, color='blue')
        
    # Set limits for the axes
    #ax.set_xlim([-scale, scale])
    ax.set_xlim([0, scale])
    ax.set_ylim([-scale, scale])
    ax.set_zlim([-scale, scale])

    # Set labels and title
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Normalized 3D Vector Plot')

    plt.show()

plot_EH_3d_vectors(Es[-1, ::2, :], Hs[-1, ::2, :])